给你一个字符串 s 和一个字符规律 p，请你来实现一个支持 '.' 和 '*' 的正则表达式匹配。
```
'.' 匹配任意单个字符
'*' 匹配零个或多个前面的那一个元素
```
所谓匹配，是要涵盖 整个 字符串 s的，而不是部分字符串。

说明:

- s 可能为空，且只包含从 a-z 的小写字母。
- p 可能为空，且只包含从 a-z 的小写字母，以及字符 . 和 *。

示例 1:
```
输入:
s = "aa"
p = "a"
输出: false
解释: "a" 无法匹配 "aa" 整个字符串。
```
示例 2:
```
输入:
s = "aa"
p = "a*"
输出: true
解释: 因为 '*' 代表可以匹配零个或多个前面的那一个元素, 在这里前面的元素就是 'a'。因此，字符串 "aa" 可被视为 'a' 重复了一次。
```
示例 3:
```
输入:
s = "ab"
p = ".*"
输出: true
解释: ".*" 表示可匹配零个或多个（'*'）任意字符（'.'）。
```
示例 4:
```
输入:
s = "aab"
p = "c*a*b"
输出: true
解释: 因为 '*' 表示零个或多个，这里 'c' 为 0 个, 'a' 被重复一次。因此可以匹配字符串 "aab"。
```
示例 5:
```
输入:
s = "mississippi"
p = "mis*is*p*."
输出: false
```

来源：力扣（LeetCode）
链接：https://leetcode-cn.com/problems/regular-expression-matching
著作权归领扣网络所有。商业转载请联系官方授权，非商业转载请注明出处。

In [ ]:
# my solution1 , 暴力穷举法, 好的我认栽了
class Solution(object):
    def isMatch(self, s, p):
        """
        :type s: str
        :type p: str
        :rtype: bool
        """
        index = 0
        n_undefined = ''
        n_count = 0
        
        for c in s:
            while(index + 1 < len(p) and p[index+1] == '*'):
                if p[index] == '.' or p[index] == c:
                    n_undefined = p[index]
                    index += 2
                    break
                index += 2
            if n_undefined:
                if n_undefined == '.':
                    break
                if n_undefined == c:
                    n_count += 1
                else:
                    while(index < len(p) and p[index] == n_undefined and n_count):
                        index += 1
                        n_count -= 1
                    if index >= len(p):
                        return False
                    if p[index] == '.' or p[index] == c:
                        index += 1
                        n_undefined = ''
                        n_count = 0
                    else:
                        return False
            else: 
                if index >= len(p):
                    return False
                if p[index] == '.' or p[index] == c:
                    index += 1
#         print(index)
        while(index < len(p)):
            if index + 1 < len(p) and p[index+1] == '*':
                index += 2
                continue
            if (p[index] == n_undefined or p[index] == '.') and n_count > 0:
                index += 1
                n_count -= 1
                continue
            return False

        return True

In [ ]:
# my solution2 , dynamic programming idea from Jake
class Solution(object):
    def isMatch(self, s, p):
        """
        :type s: str
        :type p: str
        :rtype: bool
        """
        match = [[False for _ in range(len(p)+1)] for _ in range(len(s)+1)]
        match[0][0] = True

        for i in range(len(s)+1):
            for j in range(1, len(p)+1):
                pattern = p[j-1]
                
                if pattern == '*':
                    star = p[j-2]
                    match[i][j] = match[i][j - 2] or (
                        i > 0 and match[i-1][j] and (star == s[i-1] or star == '.'))

                else:
                    match[i][j] = i>0 and match[i-1][j-1] and pattern in {s[i-1], '.'}
        return match[-1][-1]

In [ ]:
# my solution3, idea from offical
class Solution(object):
    def isMatch(self, s, p):
        """
        :type s: str
        :type p: str
        :rtype: bool
        """
        if not p:
            return not s
        first_match = bool(s) and p[0] in {s[0], '.'}
        
        if len(p) > 1 and p[1] == '*':
            return (first_match and self.isMatch(s[1:], p[:]) or (self.isMatch(s[:], p[2:])))
        else:
            return first_match and self.isMatch(s[1:], p[1:])

In [ ]:
# Jake's solution, use the dynamic programming
class Solution(object):
    def isMatch(self, s, p):

        # matched[i][j] is True if the first i chars of s (i.e. s[:i]) matches the first j chars of p (i.e. p[:j])
        matched = [[False for _ in range(len(p)+1)] for _ in range(len(s)+1)]
        # no pattern (j=0) matches no string and not any non-zero string
        matched[0][0] = True

        for i in range(len(s)+1):
            for j in range(1, len(p)+1):
                pattern = p[j-1]

                if pattern == '.':      # dot matches any last character of s
                    matched[i][j] = (i != 0 and matched[i-1][j-1])

                elif pattern == '*':    # either ignore last 2 chars of p, or ignore last char of s provided it
                    star = p[j-2]       # matches the star char
                    matched[i][j] = matched[i][j-2] or (
                        i > 0 and matched[i-1][j] and (star == s[i-1] or star == '.'))

                else:                   # pattern must match the last character of s
                    matched[i][j] = (i != 0 and matched[i-1]
                                     [j-1] and s[i-1] == pattern)
#         pp = pprint.PrettyPrinter()
#         pp.pprint(matched)
        return matched[-1][-1]

In [ ]:
import pdb
# test
input = set()
# input.add(("aa", "aac*c*a", False))
# input.add(("ab", ".*", True))

input.add(("a", "aaa", False))
solution = Solution()
for i in input:
    s, p, result = i
    if result != solution.isMatch(s, p):
        print(s, p, "Wrong", "result:{}".format(result))
#     print(s, p)
#     solution.isMatch(s, p)
#     pdb.set_trace()
# input